# Output Parsers

## Imports

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.output_parsers import CommaSeparatedListOutputParser, DatetimeOutputParser, OutputFixingParser, PydanticOutputParser
from langchain.schema.output_parser import StrOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import load_prompt

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Instantiate Model

In [3]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_tokens=300,
                        temperature=0.7)

## Instantiate Output Parser

In [4]:
output_parser = CommaSeparatedListOutputParser()

In [5]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [6]:
output_parser.parse("red, blue, white")

['red', 'blue', 'white']

## Prompts

In [7]:
human_template = "{request} /n {format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template=human_template)

In [8]:
human_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], template='{request} /n {format_instructions}'))

In [9]:
chat_prompt = ChatPromptTemplate.from_messages(messages=[human_prompt])

In [10]:
chat_prompt

ChatPromptTemplate(input_variables=['format_instructions', 'request'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], template='{request} /n {format_instructions}'))])

In [11]:
chat_prompt.format_prompt(request='give me 5 characteristics of dogs',
                          format_instructions=output_parser.get_format_instructions()).to_messages()

[HumanMessage(content='give me 5 characteristics of dogs /n Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')]

In [12]:
model_request = chat_prompt.format_prompt(request='give me 5 characteristics of dogs',
                                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [13]:
output_parser.parse((chat_model.invoke(model_request).content))

['loyal', 'affectionate', 'intelligent', 'playful', 'trainable']

## Define the chains using LCEL (LangChain Expression Language)

In [14]:
chain = chat_prompt | chat_model | CommaSeparatedListOutputParser()

In [15]:
response = chain.invoke({'format_instructions': output_parser.get_format_instructions(),
                         'request': 'give me 5 characteristics of dogs'})

In [16]:
print(response)

['loyal', 'friendly', 'intelligent', 'playful', 'protective']


## Datetime Parser

In [17]:
datetime_parser = DatetimeOutputParser()

In [18]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

system_prompt = SystemMessagePromptTemplate.from_template(template="You always reply to questions only in datetime patterns.")

In [19]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [20]:
chat_prompt

ChatPromptTemplate(input_variables=['format_instructions', 'request'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You always reply to questions only in datetime patterns.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'request'], template='{request}\n{format_instructions}'))])

In [21]:
chat_prompt.format_prompt(request="When was the 13th Amendment ratified in the US?",
                          format_instructions=datetime_parser.get_format_instructions())

ChatPromptValue(messages=[SystemMessage(content='You always reply to questions only in datetime patterns.'), HumanMessage(content="When was the 13th Amendment ratified in the US?\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1417-05-23T02:49:27.892426Z, 1275-07-09T16:35:31.732719Z, 0757-05-29T20:02:09.621975Z\n\nReturn ONLY this string, no other words!")])

In [22]:
model_prompt = chat_prompt.format_prompt(request="When was the 13th Amendment ratified in the US?",
                          format_instructions=datetime_parser.get_format_instructions()).to_messages()

In [23]:
model_prompt

[SystemMessage(content='You always reply to questions only in datetime patterns.'),
 HumanMessage(content="When was the 13th Amendment ratified in the US?\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1149-04-07T23:12:39.946121Z, 0557-03-18T20:09:13.069260Z, 0004-05-05T06:12:48.077481Z\n\nReturn ONLY this string, no other words!")]

In [25]:
datetime_parser.parse(chat_model.invoke(model_prompt).content)

datetime.datetime(1865, 12, 6, 0, 0)

## OutputFixingParser

In [27]:
new_parser = OutputFixingParser.from_llm(parser=datetime_parser, llm=chat_model)

In [29]:
new_parser.parse(chat_model.invoke(model_prompt).content)

datetime.datetime(1865, 12, 6, 0, 0)

In [32]:
chat_model.invoke(model_prompt).content

'1865-12-18T00:00:00.000000Z'

In [33]:
new_parser.parse('1865-12-18T00:00:00.000000Z')

datetime.datetime(1865, 12, 18, 0, 0)

## Pydantic Output Parser

In [38]:
class Scientist(BaseModel):

    name: str = Field(description='Name of a Scientist')
    discoveries: list = Field(description="Python list of discoveries")

In [39]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [40]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "Name of a Scientist", "type": "string"}, "discoveries": {"title": "Discoveries", "description": "Python list of discoveries", "type": "array", "items": {}}}, "required": ["name", "discoveries"]}
```


In [41]:
human_prompt = HumanMessagePromptTemplate.from_template(template="{request}\n{format_instructions}")

In [42]:
chat_prompt = ChatPromptTemplate.from_messages(messages=[human_prompt])

In [43]:
request = chat_prompt.format_prompt(request="Tell me a famous scientist",
                                    format_instructions=parser.get_format_instructions()).to_messages()

In [44]:
request

[HumanMessage(content='Tell me a famous scientist\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "Name of a Scientist", "type": "string"}, "discoveries": {"title": "Discoveries", "description": "Python list of discoveries", "type": "array", "items": {}}}, "required": ["name", "discoveries"]}\n```')]

In [46]:
print(chat_model.invoke(request).content)

```json
{
  "name": "Albert Einstein",
  "discoveries": [
    "Theory of Relativity",
    "Photoelectric Effect",
    "Mass-Energy Equivalence (E=mc²)",
    "Brownian Motion"
  ]
}
```


In [48]:
parser.parse(chat_model.invoke(request).content)

Scientist(name='Albert Einstein', discoveries=['Theory of Relativity', 'Photoelectric Effect', 'Mass-energy equivalence (E=mc²)', 'Brownian Motion', 'Bose-Einstein Statistics'])

 ## Saving and Loading Prompts

In [49]:
myprompt = PromptTemplate(template="Tell me a fact about {planet}",
                          input_variables=["planet"])

In [53]:
myprompt.save(".\output\myprompt.json")

In [36]:
loaded_prompt = load_prompt(".\output\myprompt.json")
loaded_prompt

PromptTemplate(input_variables=['planet'], input_types={}, partial_variables={}, template='Tell me a fact about {planet}')